## 웹 문서(WebBaseLoader)

<br>

### `WebBaseLoader`
- `WebBaseLoader`는 웹 기반 문서를 로드하는 로더
- `bs4` 라이브러리를 사용하여 웹 페이지를 파싱
  - `bs4.SoupStrainer`를 사용하여 파싱할 요소를 지정
  - `bs_kwargs` 매개변수를 사용하여 `bs4.SoupStrainer`의 추가적인 인수를 지정

In [1]:
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")

문서의 수: 1


In [8]:
docs[0].page_content[:100]

"\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세"

<br>

* SSL 인증 오류를 우회하려면, `verify` 옵션을 설정

In [ ]:
loader.requests_kwargs = {"verify": False}

docs = loader.load()

<br>

#### 여러 웹 페이지 로드
- `web_paths`에 리스트 전달

```python
loader = WebBaseLoader(
    web_paths=[
        "https://n.news.naver.com/article/437/0000378416",
        "https://n.news.naver.com/mnews/hotissue/article/092/0002340014?type=series&cid=2000063",
    ],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

# 데이터 로드
docs = loader.load()

# 문서 수 확인
print(len(docs))
```

<br>

- **`requests_per_second` : 초당 요청수 설정**

<br>

```python
import nest_asyncio
nest_asyncio.apply()

loader.requests_per_second = 1
docs = loader.aload()
```

<br>

<hr>

<br>

## 텍스트(`TextLoader`)


In [9]:
from langchain_community.document_loaders import TextLoader

In [10]:
loader = TextLoader("data/appendix-keywords.txt")

docs = loader.load()
print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

문서의 수: 1

[메타데이터]

{'source': 'data/appendix-keywords.txt'}

========= [앞부분] 미리보기 =========

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어


<br>

### 파일 인코딩 자동 감지
- **`silent_errors`**: 디렉토리로더에 `silent_errors` 매개변수를 전달하여 로드할 수 없는 파일을 건너뛰고 로드 프로세스를 계속할 수 있음
- **`autodetect_encoding`**: 로더 클래스에 자동 감지 인코딩을 전달하여 실패하기 전에 파일 인코딩을 자동으로 감지하도록 요청

In [11]:
from langchain_community.document_loaders import DirectoryLoader

In [12]:
path = "data/"

text_loader_kwargs = {"autodetect_encoding": True}

loader = DirectoryLoader(
    path,
    glob="**/*.txt",
    loader_cls=TextLoader,
    silent_errors=True,
    loader_kwargs=text_loader_kwargs,
)
docs = loader.load()

In [13]:
doc_sources = [doc.metadata["source"] for doc in docs]
doc_sources

['data\\appendix-keywords-CP949.txt',
 'data\\appendix-keywords-EUCKR.txt',
 'data\\appendix-keywords-utf8.txt',
 'data\\appendix-keywords.txt',
 'data\\chain-of-density.txt',
 'data\\reference.txt']

<br>

<hr>

<br>

## JSON

<br>



In [15]:
import json
from pathlib import Path
from pprint import pprint

In [19]:
file_path = "data/people.json"
data = json.loads(Path(file_path).read_text())

pprint(data[:3])

[{'address': {'city': '서울', 'street': '312번지', 'zipCode': '83795'},
  'age': 31,
  'carOwnership': True,
  'hobbies': ['요리', '음악 감상', '사진 촬영'],
  'isMarried': True,
  'name': '박시우',
  'phoneNumbers': ['483-4639-1933', '947-4179-7976']},
 {'address': {'city': '서울', 'street': '877번지', 'zipCode': '36780'},
  'age': 31,
  'carOwnership': True,
  'hobbies': ['여행', '음악 감상', '등산'],
  'isMarried': False,
  'name': '정수아',
  'phoneNumbers': ['337-5721-3227', '387-3768-9586']},
 {'address': {'city': '서울', 'street': '175번지', 'zipCode': '89067'},
  'age': 43,
  'carOwnership': True,
  'hobbies': ['등산', '독서', '게임'],
  'isMarried': False,
  'name': '최도윤',
  'phoneNumbers': ['354-5563-4638', '471-9212-1826']}]


<br>

### `JSONLoader`

```bash
$ pip install jq
```

In [23]:
from langchain_community.document_loaders import JSONLoader

In [24]:
loader = JSONLoader(
    file_path="data/people.json",
    jq_schema=".[].phoneNumbers",
    text_content=False,
)

docs = loader.load()

pprint(docs)

[Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\Sol\\Python\\LangChain\\data\\people.json', 'seq_num': 1}, page_content='["483-4639-1933", "947-4179-7976"]'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\Sol\\Python\\LangChain\\data\\people.json', 'seq_num': 2}, page_content='["337-5721-3227", "387-3768-9586"]'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\Sol\\Python\\LangChain\\data\\people.json', 'seq_num': 3}, page_content='["354-5563-4638", "471-9212-1826"]'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\Sol\\Python\\LangChain\\data\\people.json', 'seq_num': 4}, page_content='["468-2796-2152", "922-5760-7030"]'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\Sol\\Python\\LangChain\\data\\people.json', 'seq_num': 5}, page_content='["751-2823-8259", "722-7267-9516"]'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\Sol\\Python\\LangChain\\data\\people.json',

<br>

<hr>

<br>

## [Arxiv](https://arxiv.org/)
- arXiv은 물리학, 수학, 컴퓨터 과학, 정량 생물학, 정량 금융, 통계, 전기공학 및 시스템 과학, 경제학 분야의 200만 편의 학술 논문을 위한 오픈 액세스 아카이브

<br>

```bash
$ pip install -qU langchain-community arxiv pymupdf
```

<br>

#### 1. 객체 생성
- `model` 객체를 인스턴스화하고 문서를 로드

In [5]:
from langchain_community.document_loaders import ArxivLoader

- `Query` 에 검색하고자 하는 논문의 주제를 입력
- `load_max_docs` : 최대 문서 수

In [ ]:
loader = ArxivLoader(
    query="Chain of thought",
    load_max_docs=2,
    load_all_available_meta=True,  # 메타데이터 전체 로드 여부
)

In [8]:
docs = loader.load()
# docs

In [9]:
docs[0].metadata

{'Published': '2024-10-04',
 'Title': 'Understanding Reasoning in Chain-of-Thought from the Hopfieldian View',
 'Authors': 'Lijie Hu, Liang Liu, Shu Yang, Xin Chen, Zhen Tan, Muhammad Asif Ali, Mengdi Li, Di Wang',
 'Summary': "Large Language Models have demonstrated remarkable abilities across various tasks, with Chain-of-Thought (CoT) prompting emerging as a key technique to enhance reasoning capabilities. However, existing research primarily focuses on improving performance, lacking a comprehensive framework to explain and understand the fundamental factors behind CoT's success. To bridge this gap, we introduce a novel perspective grounded in the Hopfieldian view of cognition in cognitive neuroscience. We establish a connection between CoT reasoning and key cognitive elements such as stimuli, actions, neural populations, and representation spaces. From our view, we can understand the reasoning process as the movement between these representation spaces. Building on this insight, we 

<br>

### 논문 요약
- **`get_summaries_as_docs()`**

In [10]:
docs = loader.get_summaries_as_docs()

print(docs[0].page_content)

Large Language Models have demonstrated remarkable abilities across various tasks, with Chain-of-Thought (CoT) prompting emerging as a key technique to enhance reasoning capabilities. However, existing research primarily focuses on improving performance, lacking a comprehensive framework to explain and understand the fundamental factors behind CoT's success. To bridge this gap, we introduce a novel perspective grounded in the Hopfieldian view of cognition in cognitive neuroscience. We establish a connection between CoT reasoning and key cognitive elements such as stimuli, actions, neural populations, and representation spaces. From our view, we can understand the reasoning process as the movement between these representation spaces. Building on this insight, we develop a method for localizing reasoning errors in the response of CoTs. Moreover, we propose the Representation-of-Thought (RoT) framework, which leverages the robustness of low-dimensional representation spaces to enhance the

<br>

<hr>

<br>

## `LlamaParser`
- `LlamaParse` : LlamaIndex에서 개발한 문서 파싱 서비스로, 대규모 언어 모델(LLM)을 위해 특별히 설계
  - PDF, Word, PowerPoint, Excel 등 다양한 문서 형식 지원
  - 자연어 지시를 통한 맞춤형 출력 형식 제공
  - 복잡한 표와 이미지 추출 기능
  - JSON 모드 지원
  - 외국어 지원
- 사용자는 무료로 하루 1,000페이지를 처리할 수 있으며, 유료 플랜을 통해 추가 용량을 확보
  
<br>

```bash
$ pip install llama-index-core llama-parse llama-index-readers-file
```

<br>

- API 키 설정 : API 키를 발급 후 `.env` 파일에 `LLAMA_CLOUD_API_KEY` 에 설정

<br>


In [18]:
import os
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import nest_asyncio

nest_asyncio.apply()

- 파서 설정

In [3]:
parser = LlamaParse(
    result_type="markdown",  # "markdown"과 "text" 사용 가능
    num_workers=8,  # worker 수 (기본값: 4)
    verbose=True,
    language="ko",
)

- `SimpleDirectoryReader`를 사용하여 파일 파싱

In [4]:
file_extractor = {".pdf": parser}

documents = SimpleDirectoryReader(
    input_files=["data/SPRI_AI_Brief_2023년12월호_F.pdf"],
    file_extractor=file_extractor,
).load_data()


Started parsing the file under job_id cbfc58a0-c066-46c1-b919-8d929fb2a72f


In [6]:
len(documents)

23

In [16]:
documents[0].text

'# SPRiAl Brief\n\n# 인공지능 산업의 최신동향\n\n# 2023년 12월호\n\n# SPRi소프트웨어정책연구소\n\n# Software Policy &#x26; Research Institute\n'

<br>

#### `LlamaIndex` $\rightarrow$  LangChain Document 로 변환

In [13]:
docs = [doc.to_langchain_format() for doc in documents]

In [15]:
docs[0].page_content

'# SPRiAl Brief\n\n# 인공지능 산업의 최신동향\n\n# 2023년 12월호\n\n# SPRi소프트웨어정책연구소\n\n# Software Policy &#x26; Research Institute\n'

<br>

### MultiModal Model로 파싱
- `use_vendor_multimodal_model`: 멀티모달 모델 사용 여부를 지정
  - `True`로 설정하면 외부 벤더의 멀티모달 모델을 사용
- `vendor_multimodal_model_name`: 사용할 멀티모달 모델의 이름을 지정. (여기서는 "openai-gpt4o")
- `vendor_multimodal_api_key`: 멀티모달 모델 API 키 (환경 변수에서 OpenAI API 키)
- `result_type`: 파싱 결과의 형식을 지정 
- `language`: 파싱할 문서의 언어를 지정
- `skip_diagonal_text`: 대각선 텍스트를 건너뛸지 여부를 결정
- `page_separator`: 페이지 구분자를 지정
- **`user_prompt` : 사용자 정의 파싱 지침**


<br>

- 사용자 정의 파싱 지침

In [28]:
user_prompt  = (
    "You are parsing a brief of AI Report. Please extract tables in markdown format."
)

- 파서 생성

In [29]:
documents = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
    result_type="markdown",
    language="ko",
    user_prompt=user_prompt 
    # skip_diagonal_text=True,
    # page_separator="\n=================\n"
)

- parsing 된 결과


In [30]:
parsed_docs = documents.load_data(file_path="data/SPRI_AI_Brief_2023년12월호_F.pdf")

Started parsing the file under job_id 9c936777-d586-41bd-b511-b8c7f35ab9fe


- langchain 도큐먼트로 변환

In [31]:
docs = [doc.to_langchain_format() for doc in parsed_docs]

In [34]:
docs[1].page_content

'\n2023년 12월호\n\n# CONTENTS\n\n## I. 인공지능 산업 동향 브리프\n\n1. 정책/법제\n   - 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ........................................ 1\n   - G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의 ........................................ 2\n   - 영국 AI 안전성 정상회의에 참가한 28개국, AI 위협에 공동 대응 선언 ........................................ 3\n   - 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각 ................................................. 4\n   - 미국 연방거래위원회, 저작권침해 소비자 보호와 경쟁 촉진의 AI 의견서 제출 .......................... 5\n   - EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항 .............................................................. 6\n\n2. 기업/산업\n   - 미국 프런티어 모델 그룹, 1,000만 달러 규모의 AI 안전 기금 조성 ............................................. 7\n   - 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개 ..................................................... 8\n   - 앤트로픽 클라우드, 최신 LLM ‘동의치원인 2.0’ 공개 ................................................................. 9\n   - 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 .........................................

<br>

<hr>